# Exploratory Data Analysis (EDA) at Scale with Dask

In [ ]:
!pip install --upgrade "dask-cloudprovider[azure]"

In [1]:
from azureml.core import Workspace

ws = Workspace.from_config()
ws

Workspace.create(name='AzureML', subscription_id='6560575d-fa06-4e7d-95fb-f962e74efd7a', resource_group='cody-eastus-rg')

In [2]:
import git
from pathlib import Path

# get root of git repo
prefix = Path(git.Repo(".", search_parent_directories=True).working_tree_dir)

In [3]:
from azureml.core import Environment 
from dask.distributed import Client
from dask_cloudprovider import AzureMLCluster

env = Environment.from_conda_specification("dask-tutorial", prefix.joinpath("environments", "dask.yml"))
cluster = AzureMLCluster(ws, vm_size="STANDARD_DS13_V2", environment_definition=env, initial_node_count=20, scheduler_idle_timeout=1200)

c = Client(cluster)
c

WARNING - If 'script' has been provided here and a script file name has been specified in 'run_config', 'script' provided in ScriptRunConfig initialization will take precedence.
WARNING - If 'arguments' has been provided here and arguments have been specified in 'run_config', 'arguments' provided in ScriptRunConfig initialization will take precedence.
...................................................

WARNING - If 'script' has been provided here and a script file name has been specified in 'run_config', 'script' provided in ScriptRunConfig initialization will take precedence.


Client Scheduler: tcp://localhost:9002 Dashboard: http://localhost:9001,Cluster Workers: 1 Cores: 8 Memory: 59.08 GB


In [4]:
cluster

In [5]:
container_name = "isdweatherdatacontainer"

storage_options = {"account_name": "azureopendatastorage"}

In [6]:
from adlfs import AzureBlobFileSystem

fs = AzureBlobFileSystem(**storage_options)
fs

In [7]:
%%time
files = fs.glob(f"{container_name}/ISDWeather/year=*/month=*/*.parquet")
files = [f"az://{file}" for file in files]
len(files)

CPU times: user 11.2 s, sys: 574 ms, total: 11.8 s
Wall time: 3min 54s


1232

In [8]:
files[-5:]

['az://isdweatherdatacontainer/ISDWeather/year=2020/month=9/part-00003-tid-1578165671371548424-dff7e310-9776-4fe4-a52f-9ef0381fafb9-2463-9.c000.snappy.parquet',
 'az://isdweatherdatacontainer/ISDWeather/year=2020/month=9/part-00004-tid-1578165671371548424-dff7e310-9776-4fe4-a52f-9ef0381fafb9-2469-9.c000.snappy.parquet',
 'az://isdweatherdatacontainer/ISDWeather/year=2020/month=9/part-00005-tid-1578165671371548424-dff7e310-9776-4fe4-a52f-9ef0381fafb9-2466-9.c000.snappy.parquet',
 'az://isdweatherdatacontainer/ISDWeather/year=2020/month=9/part-00006-tid-1578165671371548424-dff7e310-9776-4fe4-a52f-9ef0381fafb9-2465-9.c000.snappy.parquet',
 'az://isdweatherdatacontainer/ISDWeather/year=2020/month=9/part-00007-tid-1578165671371548424-dff7e310-9776-4fe4-a52f-9ef0381fafb9-2470-9.c000.snappy.parquet']

In [9]:
import dask.dataframe as dd

In [10]:
%%time
ddf = dd.read_parquet(files, storage_options=storage_options, engine="pyarrow", blocksize="1GB")
ddf

CPU times: user 3.53 s, sys: 231 ms, total: 3.76 s
Wall time: 52.5 s


,usaf,wban,datetime,latitude,longitude,elevation,windAngle,windSpeed,temperature,seaLvlPressure,cloudCoverage,presentWeatherIndicator,pastWeatherIndicator,precipTime,precipDepth,snowDepth,stationName,countryOrRegion,p_k,year,day,version
npartitions=1232,,,,,,,,,,,,,,,,,,,,,,
,object,object,datetime64[ns],float64,float64,float64,int32,float64,float64,float64,object,int32,int32,float64,float64,float64,object,object,object,int32,int32,float64
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


In [12]:
ddf = ddf.repartition(npartitions=128).persist()

In [13]:
%%time
len(ddf)

CPU times: user 606 ms, sys: 117 ms, total: 723 ms
Wall time: 5min 43s


1506802504

In [14]:
%%time
len(ddf)

CPU times: user 155 ms, sys: 30.4 ms, total: 186 ms
Wall time: 1min 18s


1506802504

In [15]:
%%time
ddf.describe().compute()

CPU times: user 3.77 s, sys: 142 ms, total: 3.91 s
Wall time: 4min 6s


,latitude,longitude,elevation,windAngle,windSpeed,temperature,seaLvlPressure,presentWeatherIndicator,pastWeatherIndicator,precipTime,precipDepth,snowDepth,year,day,version
count,1.506802e+09,1.506803e+09,1.506803e+09,1.210044e+09,1.256867e+09,1.478103e+09,5.394822e+08,1.368751e+08,5.570711e+07,3.204236e+08,3.204236e+08,8.952745e+06,1.506803e+09,1.506803e+09,1.506803e+09
mean,3.737169e+01,-3.929356e+01,3.967938e+02,1.651766e+02,3.433093e+00,1.211181e+01,1.014643e+03,2.996899e+01,4.229705e+00,9.667262e+00,1.149292e+03,1.748470e+01,2.014203e+03,1.570473e+01,1.000000e+00
std,2.167534e+01,7.860413e+01,6.299284e+02,1.144986e+02,2.861140e+00,1.244856e+01,9.340864e+00,2.937689e+01,3.017572e+00,2.083517e+01,3.175199e+03,3.289746e+01,3.637312e+00,8.816625e+00,0.000000e+00
min,-9.000000e+01,-1.799990e+02,-3.880000e+02,0.000000e+00,0.000000e+00,-9.130000e+01,8.600000e+02,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,2.008000e+03,1.000000e+00,1.000000e+00
25%,3.404700e+01,-9.526600e+01,4.400000e+01,8.000000e+01,2.000000e+00,8.800000e+00,1.011500e+03,1.000000e+01,2.000000e+00,1.000000e+00,0.000000e+00,1.000000e+00,2.011000e+03,8.000000e+00,1.000000e+00
50%,4.161600e+01,-7.115000e+01,2.000000e+02,1.900000e+02,3.100000e+00,1.600000e+01,1.016900e+03,5.000000e+01,5.000000e+00,1.000000e+00,0.000000e+00,1.400000e+01,2.014000e+03,1.600000e+01,1.000000e+00
75%,4.913300e+01,1.745000e+01,5.480000e+02,2.800000e+02,5.700000e+00,2.740000e+01,1.025800e+03,7.100000e+01,8.000000e+00,1.200000e+01,9.999000e+03,6.200000e+01,2.017000e+03,2.400000e+01,1.000000e+00
max,8.733300e+01,9.999990e+02,9.999000e+03,3.600000e+02,9.000000e+01,6.170000e+01,1.090000e+03,9.900000e+01,9.000000e+00,9.900000e+01,9.999000e+03,9.990000e+02,2.020000e+03,3.100000e+01,1.000000e+00


In [16]:
%%time
gbs = ddf.memory_usage(index=True, deep=True).sum().compute() / 1e9
print(f"ddf is {gbs} GBs")

ddf is 741.0 GBs
CPU times: user 286 ms, sys: 55.5 ms, total: 341 ms
Wall time: 2min 39s
